### 將推文轉成心情評分 Afinn

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from afinn import Afinn # 輿情分析

In [ ]:
# (1) Define a normal Python function and match arguments to your UDF
model = Afinn()

def score_message_py(msg):
    global model
    return model.score(msg)

In [ ]:
# (2) Register UDF function
score_message = udf(score_message_py, FloatType()) # for DataFrame transformation API
spark.udf.register("score_message", score_message_py, FloatType()) # for SQL

In [ ]:
# Get average user sentimental scores
df = spark.read.csv("hdfs://devenv/user/spark/messages/data",
                    header=True,
                    inferSchema=True)

In [ ]:
# (i) Using DF transformations
scored = df.select(df["user"], score_message(df["message"]).alias("score"))
result = scored.groupBy("user").agg(avg("score"))
result.show(10000)

In [ ]:
# (ii) Using SQL
df.createOrReplaceTempView("messages")
result = spark.sql("select user, avg(score_message(message)) from messages group by user")
result.show(10000) 